# Notebook for Airconics examples 

IMPORTANT!: ipython notebook bug requires first code cells to be run manually (i.e. not with 'Run All' Cells). After this, all cells can be rerun with 'Run all below'. 

In [4]:
import airconics
import airconics.AirCONICStools as act
%gui qt4
# Initialise the display
from OCC.Display.SimpleGui import init_display
display, start_display, add_menu, add_function_to_menu = init_display()

## Airfoils

In [ ]:
LEPoint = [0., 0., 0.] 
ChordLength = 1

AirfoilSeligName = 'dae11'

# Instantiate class to set up a generic airfoil with these basic parameters
Af1 = airconics.primitives.Airfoil(LEPoint, ChordLength=3., SeligProfile=AirfoilSeligName)
Af1_disp = display.DisplayShape(Af1.Curve, update=True);

HAf1 = Af1.Curve.GetObject()


In [ ]:
# Remove Airfoil:
display.Context.Erase(Af1_disp.GetObject().GetHandle())
Af1.Curve.GetObject().Delete()   # Not sure if this truly collects the garbage yet

# Or Clear the display:
display.EraseAll()

## Simple Lofted Wing

In [2]:
# Add NACA 4 digit airfoils to loft between:
Af2 = airconics.primitives.Airfoil([0.,0.,0.], ChordLength=3., Naca4Profile='2412')
display.DisplayShape(Af2.Curve, update=True, color='GREEN');

Af3 = airconics.primitives.Airfoil([0., 5., 0.], ChordLength=1., Naca4Profile='0012')
display.DisplayShape(Af3.Curve, update=True, color='GREEN');

Af4 = airconics.primitives.Airfoil([0., 6., 0.2], ChordLength=0.2, Naca4Profile='0012')
display.DisplayShape(Af4.Curve, update=True, color='GREEN');

surf = act.AddSurfaceLoft([Af2, Af3, Af4])

# Note that surf is a TOPO_DS Shape, and hence no surf.Shape is required for display
display.DisplayShape(surf, update=True);

In [3]:
display.EraseAll()


## Wing, Transonic Airliner

TODO: I tried making the lifting surface loft through airfoil sections as a solid loft, however the output stp file was not correct in Rhino (may be because TE and ends are not closed)

In [5]:
import airconics
import airconics.AirCONICStools as act
%gui qt4
# Initialise the display
from OCC.Display.SimpleGui import init_display
display, start_display, add_menu, add_function_to_menu = init_display()
# Import all example functional definitions for the Common Research Model (CRM) Wing:
from airconics.examples.wing_example_transonic_airliner import *

# Position of the apex of the wing
P = [0., 0., 0.]

# Class definition
NSeg = 10
ChordFactor = 1
ScaleFactor = 50

# Generate (surface building is done during construction of the class)
Wing = liftingsurface.LiftingSurface(P, mySweepAngleFunctionAirliner, 
    myDihedralFunctionAirliner, 
    myTwistFunctionAirliner, 
    myChordFunctionAirliner, 
    myAirfoilFunctionAirliner, SegmentNo=NSeg, ScaleFactor=ScaleFactor)

# Here the TOPO_DS Shape to be displayed is stored in the Lifting surface 'Shape' Attribute
from OCC.Graphic3d import Graphic3d_NOM_ALUMINIUM
display.DisplayShape(Wing.Shape, update=True, material=Graphic3d_NOM_ALUMINIUM)
for section in Wing._Sections:
    display.DisplayShape(act.make_edge(section), color='Black', update=True)
# Mirror the wing:
# W2 = act.mirror(Wing.Shape, plane='xz')
# # from OCC.AIS import AIS_WireFrame
# # display.Context.SetDisplayMode(AIS_WireFrame)
# # display.Repaint()
# display.DisplayShape(W2, update=True)

<bound method BRepOffsetAPI_ThruSections.BRepOffsetAPI_ThruSections_CriteriumWeight of <OCC.BRepOffsetAPI.BRepOffsetAPI_ThruSections; proxy of <Swig Object of type 'BRepOffsetAPI_ThruSections *' at 0x7f5779c674b0> >>


### File input/output

In [6]:
# Write Output STEP file e.g. for viewing in Rhino:
act.export_STEPFile([Wing.Shape], 'wing.stp')


1

## Tailplane, Transonic Airliner

In [ ]:
from OCC.gp import gp_Ax1, gp_Pnt, gp_Dir
from airconics.examples.tailplane_example_transonic_airliner import *

# Position of the apex of the fin
P = [36.98-0.49-0.02, 0.0, 2.395-0.141]

SegmentNo = 100
ChordFact = 1.01
ScaleFact = 21.93

Fin = liftingsurface.LiftingSurface(P, mySweepAngleFunctionFin,
                                    myDihedralFunctionFin,
                                    myTwistFunctionFin,
                                    myChordFunctionFin,
                                    myAirfoilFunctionFin,
                                    SegmentNo=SegmentNo,
                                    ChordFactor=ChordFact,
                                    ScaleFactor=ScaleFact)

#    Create the rotation axis centered at the apex point in the x direction
RotAxis = gp_Ax1(gp_Pnt(*P), gp_Dir(1, 0, 0))

# Having some problem with the fin loft: display some airfoils
# to figure out what's going on:
#    for section in Fin._Sections:
#        curve = section.Curve.GetObject()
#        curve.Scale(gp_Pnt(0., 0., 0.), ScaleFact)
#        display.DisplayShape(section.Curve, update=True)

Fin.Rotate(RotAxis, 90)
display.DisplayShape(Fin.Shape, update=True)

# Position of the apex of the tailplane
P = [43, 0.000, 1.633+0.02]

SegmentNo = 100
ChordFactor = 1.01
ScaleFactor = 17.3

TP = liftingsurface.LiftingSurface(P, mySweepAngleFunctionTP,
                                   myDihedralFunctionTP,
                                   myTwistFunctionTP,
                                   myChordFunctionTP,
                                   myAirfoilFunctionTP,
                                   SegmentNo=SegmentNo,
                                   ChordFactor=ChordFact,
                                   ScaleFactor=ScaleFact)

display.DisplayShape(TP.Shape, update=True)

TP2 = act.mirror(TP.Shape, plane='xz', copy=True)

# Note: TP2 is a TopoDS_Shape, not a wing and DisplayShape is called as:
display.DisplayShape(TP2, update=True)

## Fuselage Transonic Airliner

In [ ]:
# Adding this here so I dont have to keep scrolling down: remove later
import airconics
import airconics.AirCONICStools as act
%gui qt4
# Initialise the display
from OCC.Display.SimpleGui import init_display
display, start_display, add_menu, add_function_to_menu = init_display()


In [ ]:
Fus = airconics.fuselage_oml.Fuselage()

# display.DisplayShape(Fus.PortCurve, update=True)

L = len(Fus._SectionPlanes)
P = Fus._SectionPlanes[0]
# curve = Fus.PortCurve
display.DisplayShape(Fus.FSVUCurve, update=True)
display.DisplayShape(Fus.FSVLCurve, update=True)
display.DisplayShape(Fus.FSVMeanCurve, update=True)
type(P)
display.DisplayShape(P, update=True)


In [ ]:
Af1

In [ ]:
from OCC.GeomAdaptor import GeomAdaptor_HCurve
hcurve= GeomAdaptor_HCurve(curve.GetHandle())
type(hcurve)
hcurve.GetHandle()


def intersect_shape_by_curve(topods_shape, curve, low_parameter=0.0, hi_parameter=float("+inf")):
    """
    finds the intersection of a shape and a line
    :param shape: any TopoDS_*
    :param curve: gp_Lin
    :param low_parameter:
    :param hi_parameter:
    :return: a list with a number of tuples that corresponds to the number
    of intersections found
    the tuple contains ( gp_Pnt, TopoDS_Face, u,v,w ), respectively the
    intersection point, the intersecting face
    and the u,v,w parameters of the intersection point
    :raise:
    """
    from OCC.IntCurvesFace import IntCurvesFace_ShapeIntersector
    shape_inter = IntCurvesFace_ShapeIntersector()
    shape_inter.Load(topods_shape, 1e-6)
    shape_inter.Perform(curve, low_parameter, hi_parameter)

    with assert_isdone(shape_inter, "failed to computer shape / line intersection"):
        return (shape_inter.Pnt(1),
                shape_inter.Face(1),
                shape_inter.UParameter(1),
                shape_inter.VParameter(1),
                shape_inter.WParameter(1))

intersect_shape_by_curve(P, hcurve)


In [ ]:
# Testing stuff(remove later)

from OCC.Geom2d import Geom2d_BezierCurve
from OCC.TColgp import TColgp_Array1OfPnt2d
from OCC.Geom2d import Handle_Geom2d_Curve
from OCC.gp import gp_Pnt2d
from OCC.BRepBuilderAPI import BRepBuilderAPI_MakeEdge2d
import numpy as np 
from OCC.Bnd import Bnd_Box2d
from OCC.BRepBndLib import brepbndlib_Add

NoseLengthRatio = 0.182
TailLengthRatio = 0.293
kN = NoseLengthRatio / 0.182
tN = TailLengthRatio / 0.293
AFSVUpper = np.array([[0,                 0, 0],
                      [0,                 0, 0.3],
                      [1.395*kN,          0, 1.547],
                      [4*kN,              0, 1.686],
                      [4*kN,              0, 1.686],
                      # parallel section here
                      [22-(22-15.55)*tN,  0, 1.686],
                      [22-(22-15.55)*tN,  0, 1.686],
                      [22-(22-19.195)*tN, 0, 1.549],
                      [22,                0, 0.904]])
#        Scale:
AFSVUpper *= 2.541

# The lower contour control points
# of the fuselage in side view
AFSVLower = np.array([[0,                0,  0],
                      [0,                0, -0.3],
                      [0.947*kN,         0, -0.517],
                      [4*kN,             0, -0.654],
                      [4*kN,             0, -0.654],
                      # Parallel sides section
                      [22-(22-15.55)*tN, 0, -0.654],
                      [22-(22-15.55)*tN, 0, -0.654],
                      # Tailstrike slope section
                      [22-(22-18.787)*tN,0, -0.256],
                      [22,               0,  0.694]])
AFSVLower *= 2.541

FSVMean = (AFSVUpper + AFSVLower)/2.
FSVMeanCurve = act.points_to_BezierCurve(FSVMean)

display.DisplayShape(Fus.FSVMeanCurve, update=True)

## Random things I tried out (Needs removing)

In [ ]:
# from OCC.gp import gp_Lin
# from OCC.ShapeConstruct import shapeconstruct

# Af2 = airconics.primitives.Airfoil([0.,0.,0.], ChordLength=3., Naca4Profile='2412')

# h = Af2.curve.GetHandle()
# EP = h.EndPoint()
# SP = h.StartPoint()
# closure = gp_Lin(EP, SP)

# shapeconstruct.JoinCurves(h.GetObject(), closure.)

# display.DisplayShape(Af2.Curve, update=True)

In [ ]:
# from OCC.gp import gp_Lin, gp_Ax1, gp_Dir
# from OCC.ShapeConstruct import shapeconstruct
# from OCC.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

# Af2 = airconics.primitives.Airfoil([0.,0.,0.], ChordLength=3., Naca4Profile='2412')

# h = Af2.Curve.GetObject()
# EP = h.EndPoint()
# SP = h.StartPoint()
# closure = BRepBuilderAPI_MakeEdge(gp_Lin(gp_Ax1(EP, gp_Dir(SP.XYZ()))) ).Edge()

# display.DisplayShape(closure, update=True)
# display.DisplayShape(Af2.Curve, update=True)

In [ ]:
# # Note : This seems to make objects render in ipython notebook (only works for shapes? only works if %gui qt4 has npt)
import sys
import os

# Change this to path to pythonocc-contrib repository
pythonocc_contrib_dir = os.path.abspath('/home/pchambers/git/pythonocc-contrib')
sys.path.append(pythonocc_contrib_dir)

# Create the wing:
from airconics.examples.wing_example_transonic_airliner import *

# Position of the apex of the wing
P = (0,0,0)
NSeg = 10
ChordFactor = 1
ScaleFactor = 50
Wing = liftingsurface.LiftingSurface(P, mySweepAngleFunctionAirliner, 
    myDihedralFunctionAirliner, 
    myTwistFunctionAirliner, 
    myChordFunctionAirliner, 
    myAirfoilFunctionAirliner, SegmentNo=NSeg, ScaleFactor=ScaleFactor)

import WebServer.TornadoWeb
renderer = WebServer.TornadoWeb.TornadoWebRenderer()
renderer.DisplayShape(Wing.Shape)
renderer

In [ ]:
import airconics
import airconics.AirCONICStools as act
%gui qt4
# Initialise the display
from OCC.Display.SimpleGui import init_display
display, start_display, add_menu, add_function_to_menu = init_display()

from OCC.gp import gp_Pnt
c = act.points_to_BezierCurve([gp_Pnt(0, 1, 0), gp_Pnt(2, 5, 4)])
h = c.Copy().GetObject()
display.DisplayShape(h.GetObject(), update=True)

In [ ]:
from OCC.Geom import Handle_Geom_BezierCurve
h2 = Handle_Geom_BezierCurve(h.GetHandle())

In [ ]:
# # This cell can be used to start a freecad window, create a new document and transfer the airconics
# # Wing shape to the viewer
# import sys
# sys.path.append('/usr/lib/freecad/lib')
# import FreeCAD
# import FreeCADGui
# %gui qt
# FreeCADGui.showMainWindow()

# doc = FreeCAD.newDocument()

# import Part

# import airconics 
# # Import all example functional definitions for the Common Research Model (CRM) Wing:
# from airconics.examples.wing_example_transonic_airliner import *

# # Position of the apex of the wing
# P = [0., 0., 0.]

# # Class definition
# NSeg = 10
# ChordFactor = 1
# ScaleFactor = 50

# Wing = liftingsurface.LiftingSurface(P, mySweepAngleFunctionAirliner, 
#     myDihedralFunctionAirliner, 
#     myTwistFunctionAirliner, 
#     myChordFunctionAirliner, 
#     myAirfoilFunctionAirliner, SegmentNo=NSeg, ScaleFactor=ScaleFactor)

# w = Part.__fromPythonOCC__(Wing.Shape)
# Part.show(w)

In [ ]:
# shape = Wing.Shape
# from OCC.TopExp import TopExp_Explorer
# from OCC.TopAbs import TopAbs_FACE
# from OCC.TopoDS import TopoDS_Face
# ex = TopExp_Explorer(shape, TopAbs_FACE)
# ex.More()
# currentFace = ex.Current()